https://towardsdatascience.com/custom-named-entity-recognition-with-bert-cf1fd4510804

In [25]:
!pip install seaborn
!pip install sympy
!pip install torch torchvision torchaudio

In [2]:
%%capture
!pip install transformers

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

import transformers
from transformers import AutoTokenizer
from transformers import  DistilBertForTokenClassification

from torch.optim import AdamW

import torch
import torch.nn as nn
from torch.optim import SGD
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score


In [4]:
import pandas as pd
import numpy as np

N = 1_000
df = pd.read_csv("ner.csv").sample(frac=1)[:N]
df
#change columns names
df.rename(columns = {'text':'sentence', 'labels':'tags'}, inplace = True)

#split train, dev , test sets
df_train, df_dev, df_test = np.split(df.sample(frac=1, random_state=42),
                            [int(.8 * len(df)), int(.9 * len(df))])

In [5]:
df

,sentence,tags
34133,U.S. Secretary of State Condoleezza Rice made ...,B-org O O B-org B-per I-per O O O O B-tim O
7197,Some reports say it was hauling compressed car...,O O O O O O O O O O O O O O O O O O O O
30034,The joint force came under attack during a rai...,O O O O O O O O O O O O O O O O B-geo O O
38918,The Lebanese government previously suggested i...,O B-gpe O O O O O O O B-org O O O O O O B-gpe ...
19069,That memo became controversial after several c...,O O O O O O O O O O O B-geo O B-geo I-geo O B-...
...,...,...
41473,Authorities have called the army to assist tho...,O O O O O O O O O O O O O O O O
34873,"Hours before the ruling , Israeli Defense Mini...",O O O O O B-gpe O O B-per I-per O B-gpe O O O ...
29308,Police say at least two people were injured in...,O O O O O O O O O O O O O O O O B-tim O O O O ...
32115,"The Shepherd did so , and the Lion , having ju...",O O O O O O O O O O O O O O O O O O O O O O O


#### Custom Classes
I am now going to define three classes that will be needed for this project. The first class defines our DistilBert Model. There’s no need to build the classifiers manually on top of the pretrained language model because HuggingFace already provides us with a built-in model that contains the classifier as the last layer. This model is called DistilBertForTokenClassification.

DistilbertNer Class
The init method takes into input the dimensions of the classification, so the number of tokens that we can predict, and instantiate the pretrained model.

The forward computation simply takes the inputs_ids (the tokens) and the attention_mask (array of 0/1 that tells us if the token is a PAD or not) and returns in output a dictionary : {loss, logits}

#### DistilbertNer Class
The init method takes into input the dimensions of the classification, so the number of tokens that we can predict, and instantiate the pretrained model.

The forward computation simply takes the inputs_ids (the tokens) and the attention_mask (array of 0/1 that tells us if the token is a PAD or not) and returns in output a dictionary : {loss, logits}


In [28]:
import torch.nn as nn
from transformers import  DistilBertForTokenClassification

from transformers import AutoTokenizer, DistilBertModel

class DistilbertNER(nn.Module):
  """
  Implement NN class based on distilbert pretrained from Hugging face.
  Inputs : 
    tokens_dim : int specifyng the dimension of the classifier
  """
  
  def __init__(self, tokens_dim):
    super(DistilbertNER,self).__init__()
    
    if type(tokens_dim) != int:
            raise TypeError('Please tokens_dim should be an integer')

    if tokens_dim <= 0:
          raise ValueError('Classification layer dimension should be at least 1')

    self.pretrained = DistilBertForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels = tokens_dim) #set the output of each token classifier = unique_lables


  def forward(self, input_ids, attention_mask, labels = None): #labels are needed in order to compute the loss
    """
  Forwad computation of the network
  Input:
    - inputs_ids : from model tokenizer
    - attention :  mask from model tokenizer
    - labels : if given the model is able to return the loss value
  """

    #inference time no labels
    if labels == None:
      out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask )
      return out

    out = self.pretrained(input_ids = input_ids, attention_mask = attention_mask , labels = labels)
    return out

NerDataset Class
The second class it’s an extension of the module nn.Dataset, which enables us to create our custom dataset.

Given in input a dataframe this method will tokenize the text and match the additional generated tokens with the correct tag (as we described in the previous article). Now you can index the dataset which will return a batch of texts and labels.

In [29]:
from transformers import AutoTokenizer
import torch
class NerDataset(torch.utils.data.Dataset):
  """
  Custom dataset implementation to get (text,labels) tuples
  Inputs:
   - df : dataframe with columns [tags, sentence]
  """
  
  def __init__(self, df):
    if not isinstance(df, pd.DataFrame):
      raise TypeError('Input should be a dataframe')
    
    if "tags" not in df.columns or "sentence" not in df.columns:
      raise ValueError("Dataframe should contain 'tags' and 'sentence' columns")

     
    
    tags_list = [i.split() for i in df["tags"].values.tolist()]
    texts = df["sentence"].values.tolist()

    self.texts = [tokenizer(text, padding = "max_length", truncation = True, return_tensors = "pt") for text in texts]
    self.labels = [match_tokens_labels(text, tags) for text,tags in zip(self.texts, tags_list)]

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    batch_text = self.texts[idx]
    batch_labels = self.labels[idx]

    return batch_text, torch.LongTensor(batch_labels)


MetricsTracking Class
Sometimes it’s annoying to compute all the metrics inside the train loop, that’s why this class helps us do that. You only need to instantiate a new object and call the update method every time you make predictions over a batch.

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class MetricsTracking():
  """
  In order make the train loop lighter I define this class to track all the metrics that we are going to measure for our model.
  """
  def __init__(self):

    self.total_acc = 0
    self.total_f1 = 0
    self.total_precision = 0
    self.total_recall = 0

  def update(self, predictions, labels , ignore_token = -100):
    '''
    Call this function every time you need to update your metrics.
    Where in the train there was a -100, were additional token that we dont want to label, so remove them.
    If we flatten the batch its easier to access the indexed = -100
    '''  
    predictions = predictions.flatten()
    labels = labels.flatten()
    
    predictions = predictions[labels != ignore_token]
    labels = labels[labels != ignore_token]

    predictions = predictions.to("cpu")
    labels = labels.to("cpu")

    acc = accuracy_score(labels,predictions)
    f1 = f1_score(labels, predictions, average = "macro")
    precision = precision_score(labels, predictions, average = "macro")
    recall = recall_score(labels, predictions, average = "macro")

    self.total_acc  += acc
    self.total_f1 += f1
    self.total_precision += precision
    self.total_recall  += recall

  def return_avg_metrics(self,data_loader_size):
    n = data_loader_size
    metrics = {
        "acc": round(self.total_acc / n ,3), 
        "f1": round(self.total_f1 / n, 3), 
        "precision" : round(self.total_precision / n, 3), 
        "recall": round(self.total_recall / n, 3)
          }
    return metrics

#### Custom Methods
tags_2_labels : Method that takes a list of tags and a dictionary mapping tags to labels, and returns a list of labels associated to the original tags.

In [31]:
def tags_2_labels(tags : str, tag2idx : dict):
  '''
  Method that takes a list of tags and a dictionary mapping and returns a list of labels (associated).
  Used to create the "label" column in df from the "tags" column.
  '''
  return [tag2idx[tag] if tag in tag2idx else unseen_label for tag in tags.split()] 

tags_mapping : takes in input the tags column of a dataframe and returns : (1) a dictionary mapping tags to indexes (label) (2) dictionary mapping indexes to tags (3) the label corresponding to tag O (4) a set of unique tags encountered in the train data which will define the classifier dimension.

In [32]:
def tags_mapping(tags_series : pd.Series):
  """
  tag_series = df column with tags for each sentence.
  Returns:
    - dictionary mapping tags to indexes (label)
    - dictionary mappign inedexes to tags
    - The label corresponding to tag 'O'
    - A set of unique tags ecountered in the trainind df, this will define the classifier dimension
  """

  if not isinstance(tags_series, pd.Series):
      raise TypeError('Input should be a padas Series')

  unique_tags = set()
  
  for tag_list in df_train["tags"]:
    for tag in tag_list.split():
      unique_tags.add(tag)


  tag2idx = {k:v for v,k in enumerate(sorted(unique_tags))}
  idx2tag = {k:v for v,k in tag2idx.items()}

  unseen_label = tag2idx["O"]

  return tag2idx, idx2tag, unseen_label, unique_tags


match_tokens_labels : from the tokenized text and the original tags (which are associated with words not token) it outputs an array of tags for every single token. It associates a token with the tag of its original word./


In [33]:
def match_tokens_labels(tokenized_input, tags, ignore_token = -100):
        '''
        Used in the custom dataset.
        -100 will be tha label used to match additional tokens like [CLS] [PAD] that we dont care about. 
        Inputs : 
          - tokenized_input : tokenizer over the imput text -> {input_ids, attention_mask}
          - tags : is a single label array -> [O O O O O O O O O O O O O O B-tim O]
        
        Returns a list of labels that match the tokenized text -> [-100, 3,5,6,-100,...]
        '''

        #gives an array [ None , 0 , 1 ,2 ,... None]. Each index tells the word of reference of the token
        word_ids = tokenized_input.word_ids()

        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:

            if word_idx is None:
                label_ids.append(ignore_token)

            #if its equal to the previous word we can add the same label id of the provious or -100 
            else :
                try:
                  reference_tag = tags[word_idx]
                  label_ids.append(tag2idx[reference_tag])
                except:
                  label_ids.append(ignore_token)
              
            
            previous_word_idx = word_idx

        return label_ids

freeze_model : freezes last num_layers of a model to prevent catastrophic forgetting.

In [34]:
def freeze_model(model,num_layers = 1):
  """
  Freeze last num_layers of a model to prevent ctastrophic forgetting.
  Doesn't seem to work weel, its better to fine tune the entire netwok
  """
  for id , params in enumerate(model.parameters()):
    if id == len(list(model.parameters())) - num_layers: 
      print("last layer unfreezed")
      params.requires_grad = True
    else:
      params.requires_grad = False
  return model

train_loop : usual train loop.


In [35]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train_loop(model, train_dataset, dev_dataset, optimizer,  batch_size, epochs):
  
  train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
  dev_dataloader = DataLoader(dev_dataset, batch_size = batch_size, shuffle = True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = model.to(device)

  for epoch in range(epochs) : 
    
    train_metrics = MetricsTracking()
    total_loss_train = 0

    model.train() #train mode

    for train_data, train_label in tqdm(train_dataloader):

      train_label = train_label.to(device)
      '''
      squeeze in order to match the sizes. From [batch,1,seq_len] --> [batch,seq_len] 
      '''
      mask = train_data['attention_mask'].squeeze(1).to(device)
      input_id = train_data['input_ids'].squeeze(1).to(device)

      optimizer.zero_grad()
      
      output = model(input_id, mask, train_label)
      loss, logits = output.loss, output.logits
      predictions = logits.argmax(dim= -1) 

      #compute metrics
      train_metrics.update(predictions, train_label)
      total_loss_train += loss.item()

      #grad step
      loss.backward()
      optimizer.step()
    
    '''
    EVALUATION MODE
    '''            
    model.eval()

    dev_metrics = MetricsTracking()
    total_loss_dev = 0
    
    with torch.no_grad():
      for dev_data, dev_label in dev_dataloader:

        dev_label = dev_label.to(device)

        mask = dev_data['attention_mask'].squeeze(1).to(device)
        input_id = dev_data['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask, dev_label)
        loss, logits = output.loss, output.logits

        predictions = logits.argmax(dim= -1)     

        dev_metrics.update(predictions, dev_label)
        total_loss_dev += loss.item()
    
    train_results = train_metrics.return_avg_metrics(len(train_dataloader))
    dev_results = dev_metrics.return_avg_metrics(len(dev_dataloader))

    print(f"TRAIN \nLoss: {total_loss_train / len(train_dataset)} \nMetrics {train_results}\n" ) 
    print(f"VALIDATION \nLoss {total_loss_dev / len(dev_dataset)} \nMetrics{dev_results}\n" )   


Now let’s use everything in the main scope.

In [36]:
#create tag-label mapping
tag2idx, idx2tag , unseen_label, unique_tags = tags_mapping(df_train["tags"])

#create the label column from tag. Unseen labels will be tagged as "O"
for df in [df_train, df_dev, df_test]:
  df["labels"] = df["tags"].apply(lambda tags : tags_2_labels(tags, tag2idx))

Create mapping tags-labels

In [37]:
#original text
text = df_train["sentence"].values.tolist()

#toeknized text
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
text_tokenized = tokenizer(text , padding = "max_length" , truncation = True, return_tensors = "pt" )

#mapping token to original word
word_ids = text_tokenized.word_ids()

Tokenize text


In [38]:
model = DistilbertNER(len(unique_tags))
#Prevent Catastrofic Forgetting
#model = freeze_model(model, num_layers = 2)

#datasets
train_dataset = NerDataset(df_train)
dev_dataset = NerDataset(df_dev)

lr = 1e-2
optimizer = SGD(model.parameters(), lr=lr, momentum = 0.9)  


#MAIN
parameters = {
    "model": model,
    "train_dataset": train_dataset,
    "dev_dataset" : dev_dataset,
    "optimizer" : optimizer,
    "batch_size" : 32,
    "epochs" : 8
}

train_loop(**parameters)


100%|██████████| 25/25 [11:30<00:00, 27.63s/it]


TRAIN 
Loss: 0.03076755292713642 
Metrics {'acc': 0.803, 'f1': 0.081, 'precision': 0.076, 'recall': 0.089}

VALIDATION 
Loss 0.028992019891738892 
Metrics{'acc': 0.824, 'f1': 0.108, 'precision': 0.099, 'recall': 0.119}



100%|██████████| 25/25 [10:27<00:00, 25.12s/it]


TRAIN 
Loss: 0.02102210570126772 
Metrics {'acc': 0.834, 'f1': 0.107, 'precision': 0.119, 'recall': 0.112}

VALIDATION 
Loss 0.026418519020080567 
Metrics{'acc': 0.815, 'f1': 0.1, 'precision': 0.09, 'recall': 0.111}



100%|██████████| 25/25 [10:25<00:00, 25.03s/it]


...

TRAIN 
Loss: 0.011982917487621307 
Metrics {'acc': 0.888, 'f1': 0.398, 'precision': 0.479, 'recall': 0.376}

VALIDATION 
Loss 0.015029642134904861 
Metrics{'acc': 0.897, 'f1': 0.496, 'precision': 0.56, 'recall': 0.497}



100%|██████████| 25/25 [10:42<00:00, 25.70s/it]


TRAIN 
Loss: 0.010885549597442151 
Metrics {'acc': 0.897, 'f1': 0.46, 'precision': 0.552, 'recall': 0.429}

VALIDATION 
Loss 0.01540974959731102 
Metrics{'acc': 0.888, 'f1': 0.429, 'precision': 0.499, 'recall': 0.414}



100%|██████████| 25/25 [10:35<00:00, 25.43s/it]


TRAIN 
Loss: 0.009694424867630004 
Metrics {'acc': 0.909, 'f1': 0.518, 'precision': 0.619, 'recall': 0.486}

VALIDATION 
Loss 0.014515397772192955 
Metrics{'acc': 0.899, 'f1': 0.455, 'precision': 0.51, 'recall': 0.44}

